<a href="https://colab.research.google.com/github/olgasem10/Compling-tasks/blob/master/Hometask3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Алгоритм Symspell

In [0]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
import nltk
nltk.download('punkt')
punctuation += "«»—…“”"
punct = set(punctuation)

from sklearn.metrics import classification_report, accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
!wget https://raw.githubusercontent.com/olgasem10/Compling-tasks/master/corpus_10000.txt
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/correct_sents.txt
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/sents_with_mistakes.txt 

--2019-11-21 09:52:54--  https://raw.githubusercontent.com/olgasem10/Compling-tasks/master/corpus_10000.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23882605 (23M) [text/plain]
Saving to: ‘corpus_10000.txt.1’

corpus_10000.txt.1  100%[===================>]  22.78M  --.-KB/s    in 0.1s    

2019-11-21 09:52:54 (188 MB/s) - ‘corpus_10000.txt.1’ saved [23882605/23882605]

--2019-11-21 09:52:55--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/correct_sents.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Len

In [0]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [0]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [0]:
def normalize(text):
    
    normalized_text = [(word.strip(punctuation)) for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text

In [0]:
corpus = []
for text in open('corpus_10000.txt').read().splitlines():
    sents = sent_tokenize(text)
    norm_sents = [normalize(sent) for sent in sents]
    corpus += norm_sents

In [0]:
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [0]:
N = sum(WORDS.values())
def P(word, N=N): 
    return WORDS[word] / N

Зададим функцию для генерации удалений:

In [0]:
def edits(word):
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes    = [L + R[1:] for L, R in splits if R]
    return(deletes)

Создадим словарь, ключами в котором будут слова с удалениями, а значениями - списки всех возможных исходных слов

In [0]:
voc = {}
for sent in corpus:
  for word in sent:
    for dlt in edits(word):
      if dlt in voc:
        if not word in voc[dlt]:
            voc[dlt].append(word)
      else:
        voc[dlt] = [word]


Зададим функцию, возвращающую слово с исправленной ошибкой

In [0]:
def correction(word):
  if word in WORDS:
    corr = word
  else:
    pot_words = []
    if word in voc:
      pot_words.extend(voc[word])
    cand = edits(word)
    for cnd in cand:
      if cnd in WORDS:
        pot_words.append(cnd)
      if cnd in voc:
        pot_words.extend(voc[cnd])
    if pot_words:
      corr = max(pot_words, key = P)
    else:
      corr = word
  return corr


Оценим результат

In [0]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        predicted = cashed.get(pair[1], correction(pair[1]))
        cashed[pair[0]] = predicted
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 100:
        print(i)
        

0
100
200
300
400
500
600
700
800
900


In [0]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.7328671328671329
0.4481964696853415
0.22453198575858505


#С учетом триграмной модели

In [0]:
sentences = [['<start>', '<start>'] + sent + ['<end>'] for sent in corpus]

Зададим функцию для выявления n-грамм

In [0]:
def ngrammer(tokens, n):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [0]:
bigrams = Counter()
trigrams = Counter()
for sentence in sentences:
    bigrams.update(ngrammer(sentence, 2))
    trigrams.update(ngrammer(sentence, 3))

Зададим функцию, возвращающую список всех возможных исходных слов для слова с ошибкой

In [0]:
def pot_replacements(word):
  pot_words = []
  if word in voc:
   pot_words.extend(voc[word])
  cand = edits(word)
  for cnd in cand:
    if cnd in WORDS:
      pot_words.append(cnd)
    if cnd in voc:
      pot_words.extend(voc[cnd])
  return pot_words

Зададим функцию, добавляющую к каждому слову из списка его вероятность

In [0]:
def pot_replacements_with_probs(words):
  a = [(word, P(word)) for word in words]
  with_probs = sorted(a, key=lambda x: -x[1])
  return with_probs

Оценим результат

In [0]:
mistakes = []
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

for i in range(len(true)):
  word_pairs = align_words(true[i], bad[i])
  word_pairs = [('<start>', '<start>')] + [('<start>', '<start>')] + word_pairs

  for j in range(2, len(word_pairs)):
    pred = None
    if word_pairs[j][1] in WORDS:
      pred = word_pairs[j][1]
    else:
      predicted = pot_replacements_with_probs(pot_replacements(word_pairs[j][1]))

      second_word = word_pairs[j-1][1]
      first_word = word_pairs[j-2][1]
      bigram = first_word + ' ' + second_word

      if bigram not in bigrams:
        if predicted:
          pred = predicted[0][0]
      else:
        trigram_predicted = []
        for word, prob in predicted:
          trigram = bigram + ' ' + word
          trigram_predicted.append((word, (prob)*(1+(trigrams[trigram]/bigrams[bigram]))))
      
        if trigram_predicted:
          pred = sorted(trigram_predicted, key=lambda x: -x[1])[0][0]

    if not pred:
      pred = word_pairs[j][1]

    if pred == word_pairs[j][0]:
      correct += 1
    else:
      mistakes.append((word_pairs[j][0], word_pairs[j][1], pred))
    total += 1

    if word_pairs[j][0] == word_pairs[j][1]:
      total_correct += 1
      if word_pairs[j][0] !=  pred:
        correct_broken += 1 
    
    else:
      total_mistaken += 1
      if word_pairs[j][0] == pred:
        mistaken_fixed += 1

  if not i % 50:
    print(i)
    print(correct/total)




0
0.9333333333333333
50
0.8776167471819646
100
0.8709677419354839
150
0.8751584283903675
200
0.873977873977874
250
0.879544619904517
300
0.8756009615384616
350
0.8738550117770217
400
0.875666898631408
450
0.8735416666666667
500
0.8740713224368499
550
0.8749151391717583
600
0.8777178103315343
650
0.8786249823171595
700
0.8775590551181103
750
0.8771800540407763
800
0.8751586477443175
850
0.874313408723748
900
0.874466788543571


In [0]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8747252747252747
0.4481964696853415
0.061444814517055246


Можно отметить, что общий процент правильных слов с добавлением триграмной модели вырос, и сильно уменьшилось количество "сломанных" слов, но количество исправленных ошибок не изменилось